# Analysis 

In [4]:
from IPython.display import clear_output

!pip install altair
clear_output()

In [5]:
import pandas as pd 
import altair as alt

In [16]:
pd.set_option('display.max_columns', None)

## Get Data 

In [18]:
df = pd.read_csv('../data/games_manipulated.csv', low_memory=False)
df_teams = pd.read_csv('../data/teams.csv',low_memory=False)
df_recruits = pd.read_csv('../data/recruits.csv',low_memory=False)

In [19]:
df.head() 

,Unnamed: 0,index,id,season,week,season_type,start_date,start_time_tbd,completed,neutral_site,conference_game,attendance,venue_id,venue,excitement_index,highlights,notes,main_team,home_game_flag,team_id,opposing_team_id,team_conference,opposing_conference,points_for,points_against,point_differential,team_line_scores,opposing_line_scores,team_pregame_elo,team_postgame_elo,opponent_pregame_elo,opponent_postgame_elo,win_flag,game_that_season
0,0,1,332412579,2013,1,regular,2013-08-29T22:00:00.000Z,NaN,True,False,False,81572.0,3994.0,Williams-Brice Stadium,NaN,NaN,NaN,South Carolina,1,2579,153,SEC,ACC,27.0,10.0,17.0,"[17, 3, 7, 0]","[0, 7, 3, 0]",1759.0,1785.0,1638.0,1612.0,1,1
1,1,324,332572579,2013,3,regular,2013-09-14T23:00:00.000Z,NaN,True,False,True,81371.0,3994.0,Williams-Brice Stadium,NaN,NaN,NaN,South Carolina,1,2579,238,SEC,SEC,35.0,25.0,10.0,"[21, 7, 7, 0]","[0, 10, 0, 15]",1769.0,1777.0,1644.0,1636.0,1,3
2,2,671,332782579,2013,6,regular,2013-10-05T23:30:00.000Z,NaN,True,False,True,82313.0,3994.0,Williams-Brice Stadium,NaN,NaN,NaN,South Carolina,1,2579,96,SEC,SEC,35.0,28.0,7.0,"[14, 10, 3, 8]","[0, 7, 0, 21]",1780.0,1768.0,1353.0,1365.0,1,5
3,3,1011,333062579,2013,10,regular,2013-11-02T16:21:00.000Z,NaN,True,False,True,82111.0,3994.0,Williams-Brice Stadium,NaN,NaN,NaN,South Carolina,1,2579,344,SEC,SEC,34.0,16.0,18.0,"[14, 3, 17, 0]","[7, 3, 0, 6]",1872.0,1890.0,1634.0,1616.0,1,9
4,4,1295,333202579,2013,12,regular,2013-11-17T00:00:00.000Z,NaN,True,False,True,83853.0,3994.0,Williams-Brice Stadium,NaN,NaN,NaN,South Carolina,1,2579,57,SEC,SEC,19.0,14.0,5.0,"[3, 3, 7, 6]","[7, 7, 0, 0]",1890.0,1887.0,1675.0,1678.0,1,10


## Combine Data 

In [20]:
power_5_conf = ['Pac-12', 'Big 12', 'ACC', 'SEC', 'Big Ten']


a = df[df['team_conference'].isin(power_5_conf)]
a = a[a['game_that_season'] == 12] # Final game of reg season

# Round elo to nearest 50 for binning
a['elo_rounded'] = round(a['team_postgame_elo'] / 1) * 1
a = a.groupby(by = ['season', 'team_conference', 'elo_rounded', 'main_team', 'team_id']).size().reset_index().rename({0: 'count'}, axis = 1)

a.head()

,season,team_conference,elo_rounded,main_team,team_id,count
0,2013,ACC,1197.0,Virginia,258,1
1,2013,ACC,1316.0,Wake Forest,154,1
2,2013,ACC,1367.0,NC State,152,1
3,2013,ACC,1416.0,Syracuse,183,1
4,2013,ACC,1441.0,Boston College,103,1


In [23]:
a_final = pd.merge(left = a, right = df_teams, left_on = 'team_id', right_on = 'id')

a_final = a_final[['season', 'team_conference', 'elo_rounded', 'main_team', 'logos']]
a_final['url'] = a_final['logos'].str.get(0)
a_final.drop(columns = ['logos'], axis = 1, inplace = True)

a_final.head()

,season,team_conference,elo_rounded,main_team,url
0,2013,ACC,1197.0,Virginia,[
1,2014,ACC,1491.0,Virginia,[
2,2015,ACC,1441.0,Virginia,[
3,2016,ACC,1350.0,Virginia,[
4,2017,ACC,1408.0,Virginia,[


In [28]:
a_final[a_final['season'] == 2013].head() 

,season,team_conference,elo_rounded,main_team,url
0,2013,ACC,1197.0,Virginia,[
9,2013,ACC,1316.0,Wake Forest,[
19,2013,ACC,1367.0,NC State,[
30,2013,ACC,1416.0,Syracuse,[
40,2013,ACC,1441.0,Boston College,[


In [29]:
alt.Chart(a_final).mark_image(opacity = .90, width = 20, height = 20).encode(
    
    x = alt.X('elo_rounded', title = 'ELO Rating at Season End', scale=alt.Scale(domain=[800, 2500])), 
                                y = alt.Y('team_conference', title = 'Conference'),
                                tooltip = ['main_team', 'elo_rounded'],
                                url = 'url'
                               ).properties(height = 250, width = 300)\
.facet(facet = 'season:O', columns = 4).interactive().properties(title = 'End of Regular Season ELO Rating')

alt.FacetChart(...)